In [ ]:

import json
import os
from collections import OrderedDict

import flwr as fl
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import SequentialSampler
from tqdm import tqdm
from transformers import (AdamW, RobertaModel)
from transformers import RobertaTokenizer

from src.evalution import evaluate_result


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_PATH = "/../../../datasets"
CLIENT_FILES = [f"{DATA_PATH}/llm/client_0.jsonl", f"{DATA_PATH}/llm/client_1.jsonl", f"{DATA_PATH}/llm/client_2.jsonl"]

# Hyperparametreler
BATCH_SIZE = 16
BLOCK_SIZE = 512
EPOCHS = 5
NUM_ROUNDS = 5
LEARNING_RATE = 2e-5
MODEL_NAME = "microsoft/codebert-base"

# Tokenizer yükleme
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [ ]:

class CodeDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=512):
        self.examples = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line.strip())
                code_tokens = tokenizer.tokenize(data["code_no_comment"])[:block_size - 2]
                input_ids = tokenizer.convert_tokens_to_ids([tokenizer.cls_token] + code_tokens + [tokenizer.eos_token])
                padding_length = block_size - len(input_ids)
                input_ids += [tokenizer.pad_token_id] * padding_length
                self.examples.append((torch.tensor(input_ids), torch.tensor(data["label"])))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]


In [ ]:
class CodeBERTModel(torch.nn.Module):
    def __init__(self, model_name, num_labels=4):
        super().__init__()
        self.encoder = RobertaModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, labels=None):
        outputs = self.encoder(input_ids, attention_mask=input_ids.ne(1))[0]
        cls_output = self.dropout(outputs[:, 0, :])
        logits = self.classifier(cls_output)
        loss = torch.nn.CrossEntropyLoss()(logits, labels) if labels is not None else None
        return loss, logits

In [ ]:
def train(model, train_loader, optimizer, epochs=1, checkpoint_interval=1, checkpoint_dir="checkpoints"):
    os.makedirs(checkpoint_dir, exist_ok=True)  


    checkpoint_files = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith(".bin")])
    if checkpoint_files:
        latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1])
        print(f"En son checkpoint yüklendi: {latest_checkpoint}")
        model.load_state_dict(torch.load(latest_checkpoint))

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            input_ids, labels = [b.to(DEVICE) for b in batch]
            optimizer.zero_grad()
            loss, _ = model(input_ids, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader)}")
        if (epoch + 1) % checkpoint_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch + 1}.bin")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Checkpoint kaydedildi: {checkpoint_path}")


def test(model, test_loader):
    model.eval()
    correct, total = 0, 0
    loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            input_ids, labels = [b.to(DEVICE) for b in batch]
            loss_batch, logits = model(input_ids, labels)
            loss += loss_batch.item()
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        metrics = evaluate_result(np.array(all_labels), np.array(all_preds), None)
    return loss / total, correct / total, metrics['eval_f1'], metrics['eval_precision'], metrics['eval_recall'], \
        metrics['eval_mcc'], metrics['eval_cohen_kappa_score'], metrics['eval_gmean']

In [ ]:
class CodeBERTClient(fl.client.NumPyClient):
    def __init__(self, cid, train_loader, test_loader):
        self.model = CodeBERTModel(MODEL_NAME).to(DEVICE)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.optimizer = AdamW(self.model.parameters(), lr=LEARNING_RATE)

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train(self.model, self.train_loader, self.optimizer, epochs=EPOCHS)
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy, f1, precision, recall, mcc, kappa, gmean = test(self.model, self.test_loader)
        return float(loss), len(self.test_loader.dataset), {"accuracy": float(accuracy), "loss": float(loss),
                                                            "f1": float(f1), "precision": float(precision),
                                                            "recall": float(recall), "mcc": float(mcc),
                                                            "kappa": float(kappa), "gmean": float(gmean)}



def client_fn(cid):
    train_dataset = CodeDataset(CLIENT_FILES[int(cid)], tokenizer, BLOCK_SIZE)
    test_dataset = CodeDataset(f"{DATA_PATH}/data/test_scaled.jsonl", tokenizer, BLOCK_SIZE)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    return CodeBERTClient(cid, train_loader, test_loader)


In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
MODEL_SAVE_PATH = "codebert_federated_final.bin"


def save_final_model(parameters):
    """Federated modelin son ağırlıklarını kaydeder."""
    if parameters is None:
        print("Son model parametreleri alınamadı! Kaydetme işlemi iptal edildi.")
        return

    model = CodeBERTModel(MODEL_NAME).to("cpu")  # Modeli CPU'ya al
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in zip(model.state_dict().keys(), parameters)})
    model.load_state_dict(state_dict, strict=True)
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"Federated model başarıyla kaydedildi: {MODEL_SAVE_PATH}")


def on_fit_config_fn(server_round):
    if server_round == NUM_ROUNDS:
        print(f"Son tur {NUM_ROUNDS} tamamlandı, modelin kaydedilmesi için işaret bırakılıyor...")
    return {}


def weighted_average(metrics):
    """Federated öğrenmede ağırlıklı ortalama hesaplar."""
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    losses = [num_examples * m["loss"] for num_examples, m in metrics]
    f1s = [num_examples * m["f1"] for num_examples, m in metrics]
    pres = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    mccs = [num_examples * m["mcc"] for num_examples, m in metrics]
    kappas = [num_examples * m["kappa"] for num_examples, m in metrics]
    gmeans = [num_examples * m["gmean"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(examples), "loss": sum(losses) / sum(examples),
            "f1": sum(f1s) / sum(examples), "precision": sum(pres) / sum(examples), "mcc": sum(mccs) / sum(examples),
            "recall": sum(recalls) / sum(examples), "kappa": sum(kappas) / sum(examples),
            "gmean": sum(gmeans) / sum(examples)}


strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  
    fraction_evaluate=1.0,  
    evaluate_metrics_aggregation_fn=weighted_average,
    on_fit_config_fn=on_fit_config_fn
)


hist = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,  # 3 farklı istemci
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),  # 1 tur federated eğitim
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1},  
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1}
)

#if hist.metrics_distributed and "parameters" in hist.metrics_distributed[-1]:
#    final_parameters = hist.metrics_distributed[-1]["parameters"]
#    save_final_model(final_parameters)
#    print("Federated model başarıyla kaydedildi!")
#else:
#    print("Son turdan sonra model parametreleri alınamadı!")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-03-10 17:21:36,282	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'CPU': 1.0, 'memory': 32665718784.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 16332859392.0, 'accelerator_type:T4': 1.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flow

Son tur 5 tamamlandı, modelin kaydedilmesi için işaret bırakılıyor...


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 3)
INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 6177.94s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.0636893023413966
INFO :      		round 2: 0.08374084900337386
INFO :      		round 3: 0.09940563212798173
INFO :      		round 4: 0.11366700011479902
INFO :      		round 5: 0.10415759033326846
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.7131474103585658),
INFO :      	              (2, 0.7350597609561753),
INFO :      	              (3, 0.7310756972111554),
INFO :      	              (4, 0.7390438247011952),
INFO :      	              (5, 0.749003984063745)],
INFO :      	 'f1': [(1, 0.6694530387542599),
INFO :      	        (2, 0.7074579068912084),
INFO :      	        (3, 0.7035849833559386),
INFO :     

In [ ]:
def test_model(test_file):
    tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
    model = CodeBERTModel(MODEL_NAME).to(DEVICE)
    model.load_state_dict(torch.load("/content/checkpoints/checkpoint_epoch_5.bin", map_location=DEVICE))
    model.eval()

    test_dataset = CodeDataset(test_file, tokenizer)
    test_loader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=16)

    all_preds, all_labels = [], []
    correct, total = 0, 0
    total_loss = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids, labels = [b.to(DEVICE) for b in batch]
            loss, logits = model(input_ids, labels)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total_loss += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    res = evaluate_result(all_labels, all_preds, None)
    print(res)

In [ ]:
test_model(f"{DATA_PATH}/data/test_scaled.jsonl")

<ipython-input-21-ea8510ced5b0>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/checkpoints/checkpoint_epoch_5.bin", map_location

{'eval_f1': 0.7213580841469288, 'eval_f1_perclass': [0.5714285714285714, 0.7980769230769231, 0.8958333333333334, 0.51], 'eval_acc': 0.7310756972111554, 'eval_precision': 0.7202474689904282, 'eval_recall': 0.7310756972111554, 'eval_ROC-UAC': 0.0, 'eval_mcc': 0.5244824990263965, 'eval_cohen_kappa_score': 0.5197363758769754, 'eval_gmean': 0.7391787397987591}
